# Tutorial: Simulated Ground Telescopes

This tutorial focuses on simulating data from a ground-based telescope.  We first create a fake telescope with a synthetic focalplane located in Chile.  Then we create a synthetic observing schedule and use that to scan the sky 

In [ ]:
# TOAST interactive startup
import toast.interactive as tint
import toast.interactive.magic
%load_ext toast.interactive.magic

In [ ]:
%toast -p 1 -a

In [ ]:
# Built-in modules
import sys
import os
import datetime

# External modules
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

# TOAST
import toast
from toast.instrument_sim import plot_focalplane

# Display inline plots
%matplotlib inline

In [ ]:
# MPI communicator
world, procs, rank = toast.mpi.get_world()
comm = tint.create_comm(world, single_group=True)

In [ ]:
# Output directory
out_dir = tint.create_outdir(world, topdir="out_sim_ground")

# Fake Telescope

In [ ]:
field_of_view = 5.0 * u.degree

In [ ]:
telescope = tint.create_ground_telescope(
    comm.group_size,
    pixel_per_process=7,
    sample_rate=10.0 * u.Hz,
    width=field_of_view,
)
focalplane = telescope.focalplane

In [ ]:
# Make a plot of this focalplane layout.

detpolcol = {
    x: "red" if x.endswith("A") else "blue" for x in focalplane.detectors
}

plot_focalplane(
    focalplane=focalplane,
    width=1.3 * field_of_view,
    height=1.3 * field_of_view,
    show_labels=True,
    pol_color=detpolcol
)

# Simulated Observing Schedule


In [ ]:
schedule = None

if rank == 0:
    tdir = out_dir
    if tdir is None:
        tdir = tempfile.mkdtemp()

    sch_file = os.path.join(tdir, "ground_schedule.txt")
    toast.schedule_sim_ground.run_scheduler(
        opts=[
            "--site-name",
            telescope.site.name,
            "--telescope",
            telescope.name,
            "--site-lon",
            "{}".format(telescope.site.earthloc.lon.to_value(u.degree)),
            "--site-lat",
            "{}".format(telescope.site.earthloc.lat.to_value(u.degree)),
            "--site-alt",
            "{}".format(telescope.site.earthloc.height.to_value(u.meter)),
            "--patch",
            "small_patch,1,40,-40,44,-44",
            "--start",
            "2020-01-01 00:00:00",
            "--stop",
            "2020-01-01 02:00:00",
            "--out",
            sch_file,
        ]
    )
    schedule = toast.schedule.GroundSchedule()
    schedule.read(sch_file)
    if out_dir is None:
        shutil.rmtree(tdir)
if world is not None:
    schedule = world.bcast(schedule, root=0)

Now we use this schedule to create data from our fake observing